In [ ]:

import numpy as np
import pandas as pd
import os
import librosa
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Paths to datasets
RAV_PATH = '/kaggle/input/ravdess-emotional-speech-audio'
TESS_PATH = '/kaggle/input/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data'

def load_ravdess_data():
    """Load and prepare RAVDESS dataset"""
    label_map_ravdess = {
        '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
        '05': 'angry', '06': 'fearful', '07': 'disguist', '08': 'surprised'
    }

    ravdess_file_paths = []
    ravdess_labels = []

    for actor in os.listdir(RAV_PATH):
        actor_path = os.path.join(RAV_PATH, actor)
        for file in os.listdir(actor_path):
            if file.startswith('0'):
                file_path = os.path.join(actor_path, file)
                ravdess_file_paths.append(file_path)
                emotion = file[6:8]
                ravdess_labels.append(label_map_ravdess[emotion])

    return pd.DataFrame({'paths': ravdess_file_paths, 'emotions': ravdess_labels})

def load_tess_data():
    """Load and prepare TESS dataset"""
    tess_file_paths = []
    tess_labels = []

    for folder in os.listdir(TESS_PATH):
        folder_path = os.path.join(TESS_PATH, folder)
        label = folder[4:].lower()

        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            tess_file_paths.append(file_path)
            tess_labels.append(label)

    df = pd.DataFrame({'paths': tess_file_paths, 'emotions': tess_labels})
    df['emotions'] = df['emotions'].replace({
        'pleasant_surprise': 'surprised',
        'pleasant_surprised': 'surprised',
        'fear': 'fearful',
        'disgust': 'disguist'
    })

    return df

def extract_features(file_path):
    """Extract MEL and MFCC features from audio file"""
    data, sample_rate = librosa.load(file_path)
    mel_features = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    mfcc_features = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=13).T, axis=0)
    return np.hstack((mel_features, mfcc_features))

def plot_training_history(history):
    """Plot training history metrics"""
    plt.figure(figsize=(15, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

def clean_emotion_labels(labels):
    """Remove 'x0_' prefix from emotion labels"""
    return [label.replace('x0_', '') for label in labels]

def evaluate_model(model, X_test, y_test, encoder):
    """Comprehensive model evaluation with clean emotion labels"""
    # Get predictions
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Get emotion labels and clean them
    emotion_labels = clean_emotion_labels(encoder.get_feature_names_out())

    print("\n=== Model Evaluation ===")

    # 1. Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true_classes, y_pred_classes,
                              target_names=emotion_labels,
                              digits=3))

    # 2. Confusion Matrix
    print("\nPlotting Confusion Matrix...")
    plot_confusion_matrix(y_true_classes, y_pred_classes, emotion_labels)

    # 3. Per-class Accuracy
    print("\nCalculating Per-class Accuracy...")
    accuracies = plot_per_class_accuracy(y_true_classes, y_pred_classes, emotion_labels)

    # 4. Print per-class accuracy summary
    print("\nPer-class Accuracy Summary:")
    for emotion, acc in zip(emotion_labels, accuracies):
        print(f"{emotion}: {acc:.3f}")

    # 5. Additional metrics
    print("\nAdditional Metrics:")
    print(f"Overall Accuracy: {np.mean(y_true_classes == y_pred_classes):.3f}")

    # Return metrics for saving
    return {
        'confusion_matrix': confusion_matrix(y_true_classes, y_pred_classes),
        'per_class_accuracy': dict(zip(emotion_labels, accuracies)),
        'overall_accuracy': np.mean(y_true_classes == y_pred_classes)
    }

def plot_confusion_matrix(y_true, y_pred, labels):
    """Plot confusion matrix with proper labels"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 8))
    sns.heatmap(cm,
                xticklabels=labels,
                yticklabels=labels,
                annot=True,
                fmt='d',
                cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_per_class_accuracy(y_true, y_pred, labels):
    """Plot accuracy for each emotion class"""
    # Calculate per-class accuracy
    accuracies = []
    for i in range(len(labels)):
        mask = (y_true == i)
        acc = np.mean(y_pred[mask] == y_true[mask])
        accuracies.append(acc)

    plt.figure(figsize=(12, 6))
    sns.barplot(x=labels, y=accuracies)
    plt.title('Per-Class Accuracy')
    plt.xlabel('Emotion')
    plt.ylabel('Accuracy')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return accuracies

def main():
    # Create output directory
    OUTPUT_DIR = '/kaggle/working/emotion_recognition_export'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print("Loading datasets...")
    rav_data = load_ravdess_data()
    tess_data = load_tess_data()
    emotion_data = pd.concat([rav_data, tess_data], axis=0).reset_index(drop=True)

    print("\nExtracting features...")
    features = []
    for i, path in enumerate(emotion_data['paths']):
        if i % 100 == 0:
            print(f"Processing file {i}/{len(emotion_data)}")
        features.append(extract_features(path))
    features = np.array(features)

    # Prepare data for training
    X = features
    y = emotion_data['emotions'].to_numpy()

    # Encode labels
    encoder = OneHotEncoder()
    y_encoded = encoder.fit_transform(y.reshape(-1, 1)).toarray()

    # Split and scale data
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Reshape for TimeDistributed layer
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    # Build model
    model = Sequential([
        TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
                       input_shape=(X_train.shape[1], X_train.shape[2], 1)),
        TimeDistributed(MaxPooling1D(pool_size=2)),
        TimeDistributed(Flatten()),
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.3),
        Bidirectional(LSTM(64)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(y_train.shape[1], activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.005),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    print("\nTraining model...")
    history = model.fit(X_train, y_train,
                       validation_data=(X_test, y_test),
                       epochs=150,
                       batch_size=32,
                       verbose=1)

    # Plot training history
    print("\nPlotting training history...")
    plot_training_history(history)

    # Evaluate model
    print("\nEvaluating model...")
    evaluation_metrics = evaluate_model(model, X_test, y_test, encoder)

    # Save everything
    print("\nSaving model and artifacts...")

    # 1. Save the model
    model.save(os.path.join(OUTPUT_DIR, 'emotion_model.keras'))

    # 2. Save the encoder
    with open(os.path.join(OUTPUT_DIR, 'encoder.pkl'), 'wb') as f:
        pickle.dump(encoder, f)

    # 3. Save the scaler
    with open(os.path.join(OUTPUT_DIR, 'scaler.pkl'), 'wb') as f:
        pickle.dump(scaler, f)

    # 4. Save model configuration
    config = {
        'sample_rate': 22050,
        'mel_features': 128,
        'mfcc_features': 13,
        'total_features': 141,
        'emotions': list(encoder.get_feature_names_out())
    }
    with open(os.path.join(OUTPUT_DIR, 'model_config.pkl'), 'wb') as f:
        pickle.dump(config, f)

    # 5. Save training history
    history_dict = {
        'accuracy': history.history['accuracy'],
        'val_accuracy': history.history['val_accuracy'],
        'loss': history.history['loss'],
        'val_loss': history.history['val_loss']
    }
    with open(os.path.join(OUTPUT_DIR, 'training_history.pkl'), 'wb') as f:
        pickle.dump(history_dict, f)

    # 6. Save evaluation metrics
    with open(os.path.join(OUTPUT_DIR, 'evaluation_metrics.pkl'), 'wb') as f:
        pickle.dump(evaluation_metrics, f)

    print(f"\nTraining complete! All files saved to {OUTPUT_DIR}")
    print("\nFiles saved:")
    print("1. emotion_model.keras - The trained model")
    print("2. encoder.pkl - Label encoder")
    print("3. scaler.pkl - Feature scaler")
    print("4. model_config.pkl - Model configuration")
    print("5. training_history.pkl - Training history")
    print("6. evaluation_metrics.pkl - Evaluation metrics")

if __name__ == "__main__":
    main()